<a target="_blank" href="https://colab.research.google.com/github/michalis0/Business-Intelligence-and-Analytics/blob/master/labs/08%20-%20Regression2/walkthrough/walkthrough_08.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<h1 align="center"> WALKTHROUGH 8</h1>

<div>
<td> 
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/2b/Logo_Universit%C3%A9_de_Lausanne.svg/2000px-Logo_Universit%C3%A9_de_Lausanne.svg.png" style="padding-right:10px;width:240px;float:left"/></td>
<h2 style="white-space: nowrap">Business Intelligence and Analytics</h2></td>
<hr style="clear:both">
<p style="font-size:0.85em; margin:2px; text-align:justify">

</div>

This week we continue working on regression. Diving into more specific methods, we will show you how to choose the number of parameters, cross validation and 1-hot/label encoding.

# REGRESSION WITH CATEGORICAL VARIABLES

In [ ]:
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import seaborn as sns
import statistics
%matplotlib inline
sns.set(style="darkgrid")

## LOAD THE DATASET

The Dataset we will be working on was extracted and modified from [Kaggle](https://www.kaggle.com/mohansacharya/graduate-admissions). It gives us various information concerning students and their chances of admission.


In [ ]:
#Load the dataset
url = 'https://media.githubusercontent.com/media/michalis0/Business-Intelligence-and-Analytics/master/data/Admissions_prediction.csv'
GAD = pd.read_csv(url, sep = ";", index_col= 'Serial No.').drop_duplicates() #Graduate Admissions Data

display(GAD.head())
print(GAD.dtypes)
print("Data matrix shape: ", GAD.shape)

An overview of the columns:

 
 * `GRE Score`: GRE is a standardized admission test (out of 340)
 * `TOEFL Score`: English knowledge (out of 120)
 * `SOP`: Standard of Purpose (out of 5)
 * `LOR`: Letter of Recomendation (out of 5)
 * `CGPA`: College GPA (out of 10)
 * `RESEARCH`: Whether the applicant did research or not
 
 We will try to predict the chance of admit with the other variables.

## HANDLING CATEGORICAL VARIABLES FOR REGRESSION
First, let's focus on the `Research` column. The applicant was either active in research, or he was not. There are therefore only 2 possible values, hence we will assign 1 for YES and 0 for NO.

In [ ]:
GAD['Research'] = GAD['Research'].replace(['NO', 'YES'], [0,1])
display(GAD.head())

Here let's get a quick overview of the existing correlations between our variables using a heatmap.

In [ ]:
d = sns.heatmap(GAD.corr(numeric_only=True), annot=True)

In [ ]:
# We want to predict the chance of admission
y = GAD[['Chance of Admit']]

# With the help of the other columns
X = GAD.drop('Chance of Admit', axis = 1)

Now we need to handle the `University Ranking`. 
We will do both, one-hot and label encoding.
For label encoding, we can use Sklearn's LabelEncoder() function. 


In [ ]:
# Label encoder
X_label = X.apply(LabelEncoder().fit_transform)

In [ ]:
display(X_label.head())

For one hot encoding we use pandas get dummy function.

In [ ]:
# 1-hot encoding

# We create a DF with Dummy variables
dummies = pd.get_dummies(GAD["University Rating"])
X_hot = pd.concat([GAD, dummies], axis = 1)

# We drop the Rating column 
del X_hot["University Rating"]
del X_hot["Chance of Admit"]

# Have a look at what dummies actually looks like
dummies

In [ ]:
display(X_hot.head())

We will now build two models, one for 1-hot encoding and one for label encoding.
Let's start with label encoding:

In [ ]:
# Split Dataset
X_train_lab, X_test_lab, y_train_lab, y_test_lab = train_test_split(X_label, y, test_size=0.2, random_state=0, shuffle=True)

# Fit the model
model_lab = LinearRegression()
model_lab.fit(X_train_lab, y_train_lab)

# Calculate R2
lab_r2 = round(model_lab.score(X_train_lab, y_train_lab), 4)
print("R^2 of train set using label encoding: ", lab_r2 )

# Calculate Scores on Test-Set
label_predictions = model_lab.predict(X_test_lab)
lab_mae = mean_absolute_error(y_test_lab, label_predictions)
lab_mse = mean_squared_error(y_test_lab, label_predictions)
lab_r2_test = r2_score(y_test_lab, label_predictions)

print("MAE LAB %.2f" % lab_mae)
print("MSE LAB %.2f" % lab_mse)
print("R^2 score LAB %.4f" % lab_r2_test)

And continue with 1-hot:

In [ ]:
# Split Dataset
X_train_hot, X_test_hot, y_train_hot, y_test_hot = train_test_split(X_hot, y, test_size=0.2, random_state=0, shuffle=True)

# Fit the model
model_hot = LinearRegression()
model_hot.fit(X_train_hot, y_train_hot)

# Calculate R2
hot_r2 = round(model_hot.score(X_train_hot, y_train_hot), 4)
print("R^2 Train Score using 1-hot encoding : ", hot_r2 )

# Calculate Scores on Test-Set
hot_predictions = model_hot.predict(X_test_hot)
hot_mae = mean_absolute_error(y_test_hot, hot_predictions)
hot_mse = mean_squared_error(y_test_hot, hot_predictions)
hot_r2_test = r2_score(y_test_hot, hot_predictions)

print("MAE HOT %.2f" % hot_mae)
print("MSE HOT %.2f" % hot_mse)
print("R^2 score HOT %.4f" % hot_r2_test)

In [ ]:
print("The 1-hot encoding method yields an R^2 {}% higher than using label encoding for the train set.".format(round((hot_r2/lab_r2 -1)*100, 3)))

print("The 1-hot encoding method yields an R^2 {}% higher than using label encoding for the test set.".format(round((hot_r2_test/lab_r2_test -1)*100, 3)))

Have a look at how the coefficients change for each method:

In [ ]:
print("Intercept: ", model_hot.intercept_[0]) 
print("Intercept: ", model_lab.intercept_[0])

In [ ]:
weights_hot = model_hot.coef_.flatten()
weights_lab = model_lab.coef_.flatten()

print("Features coefficients for HOT (weigths): ", model_hot.coef_)
print("Features coefficients for LAB (weigths): ", model_lab.coef_)

# POLYNOMIAL LINEAR REGRESSION
We will now have a look at another model. We are going to use `Advertisement` data we previously used. The task is to figure out how different means of advertisement influence the amount of sales of a product.

In [ ]:
ad_df = pd.read_csv('https://media.githubusercontent.com/media/michalis0/Business-Intelligence-and-Analytics/master/data/Advertising.csv')
# view the first 5 rows 
ad_df.head()

In [ ]:
# Prepare the data for the regression
y = ad_df["Sales"]
X = ad_df.drop("Sales", axis = 1)

In [ ]:
# First let's go for a linear multivariate regression
# You should know the drill by now
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

# Performance metrics
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("Test scores: ")
print("MAE %.2f" % mae)
print("MSE %.2f" % mse)
print("R^2 %.2f" % r2)

print("Train metrics")
print("params: ", model.coef_)
print("constant: ", model.intercept_)
print("R^2 score: ", model.score(X, y))

Using polynomial regression enables you to predict the best fit line that follows the pattern(curve) of the data. It tends to increase the performance of the model. The function PolynomialFeatures generates a new feature matrix consisting of all polynomial combinations of the features with degree less than or equal to the specified degree. For example, if an input sample is two dimensional and of the form [a, b], the degree-2 polynomial features are [1, a, b, a^2, ab, b^2]. Let's try it on our model.

In [ ]:
# Polynomial regression
poly = PolynomialFeatures(2)
X = np.array(ad_df[["TV", "Radio"]])
y = np.array(ad_df["Sales"])
X_poly = poly.fit_transform(X)

LR = LinearRegression(fit_intercept=False) # We don't need fit intercept sice polynomial features function add a column of ones to the data 
LR.fit(X_poly, y)
print("params: ", LR.coef_)
print("R^2 score: ", LR.score(X_poly, y))

## OVERFITTING
Beware, adding too many features may cause overfitting.
Remember that overfitting is is the tendency of data mining procedures to tailor models to the training data, at the expense of generalization to previously unseen data points. We want to avoid this case.

In [ ]:
train_err = []
test_err = []
for f in range(1,7):
    poly = PolynomialFeatures(f)
    X_poly = poly.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=12)
    LR = LinearRegression(fit_intercept=False)
    LR.fit(X_train, y_train)
    train_err.append(mean_squared_error(y_train, LR.predict(X_train)))
    test_err.append(mean_squared_error(y_test, LR.predict(X_test)))


fig=plt.figure(figsize=(11,7))
plt.plot(range(1,7), train_err, label="train_error")
plt.plot(range(1,7), test_err, label="test_error")
plt.legend(fontsize=10)
plt.xlabel("polynomial degree")
dummy = plt.ylabel("error")


In [ ]:
from sklearn.model_selection import KFold


y = ad_df["Sales"]
X = ad_df.drop("Sales", axis = 1)

kf = KFold(n_splits = 10, random_state = None)
model = LinearRegression()

mae_cumm = []
mse_cumm = []
r2_cumm = []

for train_index, test_index in kf.split(X):
  
  X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  model.fit(X_train, y_train)
  predictions = model.predict(X_test)


  mae = mean_absolute_error(y_test, predictions)
  mse = mean_squared_error(y_test, predictions)
  r2 = r2_score(y_test, predictions)

  mae_cumm.append(mae)
  mse_cumm.append(mse)
  r2_cumm.append(r2)

  
mean_mae = sum(mae_cumm)/len(mae_cumm)
mean_mse = sum(mse_cumm)/len(mse_cumm)
mean_r2 = sum(r2_cumm)/len(r2_cumm)

print("The mean absolute error of all our folds was: ",round(mean_mae, 3))
print("The mean squared error of all our folds was: ", round(mean_mse, 3))
print("The mean accuracy of all our folds was: ", round(mean_r2, 3))


And for higher order polynomial regression:

In [ ]:
train_err = []
test_err = []

y = ad_df["Sales"]
X = ad_df.drop("Sales", axis = 1)

kf = KFold(n_splits = 10, random_state = None)
model = LinearRegression(fit_intercept = False)

for f in range(1,7):
  train = []
  test = []
  for train_index, test_index in kf.split(X):
    poly = PolynomialFeatures(f)
    X_poly = poly.fit_transform(X)
    X_train, X_test = X_poly[train_index,:], X_poly[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model.fit(X_train, y_train)
    train.append(mean_squared_error(y_train, model.predict(X_train)))
    test.append(mean_squared_error(y_test, model.predict(X_test)))

  train_err.append(statistics.mean(train))
  test_err.append(statistics.mean(test))

fig=plt.figure(figsize=(11,7))
plt.plot(range(1,7), train_err, label="train_error")
plt.plot(range(1,7), test_err, label="test_error")
plt.legend(fontsize=10)
plt.xlabel("polynomial degree")
plt.ylabel("error")
print(train_err)
print(test_err)